In [1]:
import pandas as pd
import requests
import time

In [2]:
users = pd.read_excel("data/users_enriched.xlsx")
users_filtered = users.loc[users['final_decision'] == 1]
users_filtered

,user_id,source,id,node_id,avatar_url,gravatar_id,url,html_url,followers_url,following_url,...,created_at,updated_at,is_student,is_employee,is_currently_employed,is_research_group,final_decision,note,new_entry,date
3,73616e646572,employees_github_usernames.csv,6605233.0,MDQ6VXNlcjY2MDUyMzM=,https://avatars.githubusercontent.com/u/660523...,NaN,https://api.github.com/users/73616e646572,https://github.com/73616e646572,https://api.github.com/users/73616e646572/foll...,https://api.github.com/users/73616e646572/foll...,...,2014-02-06T13:51:56Z,2021-11-19T12:23:00Z,False,1.0,1.0,0.0,1,0,False,2021-12-02
4,a-quei,ids_search_users.csv,6829836.0,MDQ6VXNlcjY4Mjk4MzY=,https://avatars.githubusercontent.com/u/682983...,NaN,https://api.github.com/users/a-quei,https://github.com/a-quei,https://api.github.com/users/a-quei/followers,https://api.github.com/users/a-quei/following{...,...,2014-03-02T08:21:10Z,2021-08-12T20:33:30Z,False,0.0,0.0,0.0,1,0,False,2021-12-02
9,ajueling,ids_search_repos.csv,20947797.0,MDQ6VXNlcjIwOTQ3Nzk3,https://avatars.githubusercontent.com/u/209477...,NaN,https://api.github.com/users/AJueling,https://github.com/AJueling,https://api.github.com/users/AJueling/followers,https://api.github.com/users/AJueling/followin...,...,2016-08-10T11:21:34Z,2021-11-11T14:52:36Z,False,1.0,1.0,0.0,1,0,False,2021-12-02
10,albertgatt,employees_github_usernames.csv,8494566.0,MDQ6VXNlcjg0OTQ1NjY=,https://avatars.githubusercontent.com/u/849456...,NaN,https://api.github.com/users/albertgatt,https://github.com/albertgatt,https://api.github.com/users/albertgatt/followers,https://api.github.com/users/albertgatt/follow...,...,2014-08-19T17:30:50Z,2021-11-03T15:58:02Z,False,1.0,1.0,0.0,1,NaN,True,2021-12-02
12,aldertzomer,ids_search_users.csv,10392411.0,MDQ6VXNlcjEwMzkyNDEx,https://avatars.githubusercontent.com/u/103924...,NaN,https://api.github.com/users/aldertzomer,https://github.com/aldertzomer,https://api.github.com/users/aldertzomer/follo...,https://api.github.com/users/aldertzomer/follo...,...,2015-01-04T19:49:33Z,2021-11-17T18:27:58Z,False,0.0,0.0,0.0,1,0,False,2021-12-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,wouter-swierstra,ids_search_repos.csv,1588245.0,MDQ6VXNlcjE1ODgyNDU=,https://avatars.githubusercontent.com/u/158824...,NaN,https://api.github.com/users/wouter-swierstra,https://github.com/wouter-swierstra,https://api.github.com/users/wouter-swierstra/...,https://api.github.com/users/wouter-swierstra/...,...,2012-03-29T19:25:32Z,2021-10-22T14:30:05Z,False,1.0,1.0,0.0,1,0,False,2021-12-02
478,wouterdroogers,ids_search_users.csv,38694532.0,MDQ6VXNlcjM4Njk0NTMy,https://avatars.githubusercontent.com/u/386945...,NaN,https://api.github.com/users/WouterDroogers,https://github.com/WouterDroogers,https://api.github.com/users/WouterDroogers/fo...,https://api.github.com/users/WouterDroogers/fo...,...,2018-04-24T13:44:53Z,2021-01-07T15:10:19Z,False,1.0,1.0,0.0,1,0,False,2021-12-02
486,yuntaoj,ids_search_users.csv,74980142.0,MDQ6VXNlcjc0OTgwMTQy,https://avatars.githubusercontent.com/u/749801...,NaN,https://api.github.com/users/yuntaoj,https://github.com/yuntaoj,https://api.github.com/users/yuntaoj/followers,https://api.github.com/users/yuntaoj/following...,...,2020-11-24T16:38:30Z,2021-03-08T10:03:10Z,False,0.0,0.0,0.0,1,0,False,2021-12-02
487,zievathustra,ids_search_repos.csv,61464979.0,MDQ6VXNlcjYxNDY0OTc5,https://avatars.githubusercontent.com/u/614649...,NaN,https://api.github.com/users/zievathustra,https://github.com/zievathustra,https://api.github.com/users/zievathustra/foll...,https://api.github.com/users/zievathustra/foll...,...,2020-02-25T14:22:18Z,2021-11-09T10:56:39Z,False,1.0,1.0,0.0,1,0,False,2021-12-02


# Faculties
* Faculty of Geosciences
* Faculty of Humanities
* Faculty of Law, Economics and Governance
* Faculty of Medicine
* Faculty of Science
* Faculty of Social and Behavioural Sciences
* Faculty of Veterinary Medicine

In [75]:
name = "Dorien Huijser"
def search_employee_url(name):
    if "nan" == name or " " not in name:
        return None
    name_url = name.replace(" ", "%20")
    url = f"https://www.uu.nl/medewerkers/RestApi/Public/search?expression=%7B%22q%22%3A%22{name_url}%22%2C%22mode%22%3A%22naam%22%7D&t=null&l=EN"
    json_nested  = requests.get(url)
    try:
        employee_url = json_nested.json()["Employees"][0]["Url"]
#         print(name, employee_url)
        return employee_url
    except Exception as e:
        return None
    
search_employee_url(name)

'DCHuijser'

In [76]:
employee_url = "EvanKesteren"
def get_employee_info(employee_url):
    if employee_url is None:
        return [None]*3
    API_link = requests.get(f"https://www.uu.nl/medewerkers/RestApi/Public/getEmployeeData?page={employee_url}&l=EN")
    API_json = API_link.json()["Employee"]
    # faculties is an array but retrieved data always had only one, so extra faculties will be discarded
    fac = API_json["Faculties"][0]["Faculty"]
    positions = [API_json["Positions"][0]["Level1"], 
                 API_json["Positions"][0]["Level2"], 
                 API_json["Positions"][0]["Level3"], 
                 API_json["Positions"][0]["Level4"], 
                 API_json["Positions"][0]["Position"]]
    info = [API_json["NameShort"], fac, positions]
    return info

get_employee_info(employee_url)

['Erik-Jan van Kesteren',
 'Social and Behavioural Sciences',
 ['Social and Behavioural Sciences', '', None, None, 'Assistant Professor']]

In [78]:
employee_urls = []
for user in users_filtered["name"]:
    result = search_employee_url(str(user))
    print(user, result)
    employee_urls.append(result)
#     time.sleep(1)
print(employee_urls)

SP None
Guanyi Chen GChen
André Jüling AJuling2
Albert Gatt AGatt
Aldert Zomer ALZomer
Alexandros Stergiou None
Ana Macanovic AMacanovic
MICE None
Anastasia Giachanou AGiachanou
Andreas Tataris ATataris
Anna-Lena Lamprecht ALLamprecht
Anna Wegmann AMWegmann
Arno Bastenhof None
aromanowski None
ASReview None
basm92 None
nan None
nan None
Bilal Farooq None
Britta Ricker BARicker
C. J. van Lissa None
CLARIAH None
Computational Geography None
Costanza None
Claudio Pierard CMPierard
D-score None
Daniel Oberski DLOberski
David Ignacio Cortes None
David Gernaat None
Derek Karssenberg DJKarssenberg
nan None
Dorien Huijser DCHuijser
Daniel Zamrsky DZamrsky
Eugene Katrukha YKatrukha
nan None
Fouad Sfarijlani None
Francisca Pessanha MFPessanhadeMenesesRibeirodosReis
FriedrichFoerster None
Kazi Injamamul Haque None
Gabriel Beckers GJLBeckers
Geertje None
nan None
Gerbrand Sterrenburg ACJGSterrenburg
Gerko Vink GVink
nan None
nan None
nan None
nan None
growthcharts None
Garrett Speed GTSpeed
Guus V

In [28]:
employee_infos = []
for employee_url in employee_urls:
    result = get_employee_info(employee_url)
    print(result)
    employee_infos.append(result)

['Casper Arends', 'Humanities', ['Humanities', 'Student Affairs, Education and Research', 'General', None, 'Student Affairs Assistant']]
['G. (Guanyi) Chen', 'Science', ['Science', 'Information and computing sciences', 'Intelligent Software Systems', 'Natural Language Processing', 'Lecturer']]
['A. (Andre) Jüling', 'UU', ['UU', '', '', '', None]]
['A. (Albert) Gatt', 'Science', ['Science', 'Information and computing sciences', 'Intelligent Software Systems', 'Natural Language Processing', 'Associate Professor']]
['A.L. (Aldert) Zomer', 'Veterinary Medicine', ['Veterinary Medicine', 'Department Biomolecular Health Sciences', 'Infectious Diseases & Immunology', ' Clinical Infectiology', 'Assistant Professor']]
[None, None, None]
['Ana Macanovic', 'Social and Behavioural Sciences', ['Social and Behavioural Sciences', 'Social Sciences', 'Sociology', None, 'PhD Candidate']]
['Elmin Omičević', 'Law, Economics and Governance', ['Law, Economics and Governance', 'Utrecht University School of La

['N. (Nima) Motamed', 'Science', ['Science', 'Information and computing sciences', 'Intelligent Software Systems', 'Intelligent Systems', 'PhD Candidate']]
[None, None, None]
[None, None, None]
[None, None, None]
[None, None, None]
['Sheean Spoel', 'Humanities', ['Humanities', 'Digital Humanities IT', None, None, 'ICT Developer']]
[None, None, None]
['Pierre-Alexandre Balland', 'Geosciences', ['Geosciences', 'Human Geography and Spatial Planning', 'Economic Geography', None, 'Associate Professor']]
['P.G. (Patrick) Finke', 'Science', ['Science', 'Mathematics', 'Mathematical Institute', 'Mathematical Modeling', 'PhD Candidate']]
[None, None, None]
[None, None, None]
['Peter Gerbrands', 'Law, Economics and Governance', ['Law, Economics and Governance', 'Utrecht University School of Economics ', 'Applied Economics', None, 'PhD Candidate']]
[None, None, None]
[None, None, None]
[None, None, None]
[None, None, None]
[None, None, None]
[None, None, None]
[None, None, None]
['L.D. (Roy) Hoiti

In [49]:
# for employee_info in employee_infos:
#     print(employee_info[1]) # this shows that there are never 2 faculties for a person even if the rest api allows it
    
columns = ["employee_url", "faculty", "level1", "level2", "level3", "level4", "position"]
flat_list = []
for employee_url, (_, faculty, positions) in zip(employee_urls, employee_infos):
    if positions is None:
        flat_list.append([employee_url, faculty, None, None, None, None, None])
    else:
        pos = [None if position == '' else position for position in positions]
        flat_list.append([employee_url, faculty, pos[0], pos[1], pos[2], pos[3], pos[4]])
flat_list

[['CFArends',
  'Humanities',
  'Humanities',
  'Student Affairs, Education and Research',
  'General',
  None,
  'Student Affairs Assistant'],
 ['GChen',
  'Science',
  'Science',
  'Information and computing sciences',
  'Intelligent Software Systems',
  'Natural Language Processing',
  'Lecturer'],
 ['AJuling2', 'UU', 'UU', None, None, None, None],
 ['AGatt',
  'Science',
  'Science',
  'Information and computing sciences',
  'Intelligent Software Systems',
  'Natural Language Processing',
  'Associate Professor'],
 ['ALZomer',
  'Veterinary Medicine',
  'Veterinary Medicine',
  'Department Biomolecular Health Sciences',
  'Infectious Diseases & Immunology',
  ' Clinical Infectiology',
  'Assistant Professor'],
 [None, None, None, None, None, None, None],
 ['AMacanovic',
  'Social and Behavioural Sciences',
  'Social and Behavioural Sciences',
  'Social Sciences',
  'Sociology',
  None,
  'PhD Candidate'],
 ['EOmicevic1',
  'Law, Economics and Governance',
  'Law, Economics and Gove

In [79]:
pd_info = pd.DataFrame(flat_list,columns=columns)
pd_info
users_filtered.reset_index(drop=True, inplace=True)
pd_merged = pd.merge(users_filtered, pd_info, left_index=True, right_index=True)
pd_merged.to_csv("data/users_labeled.csv", index=False)
pd_merged

,user_id,source,id,node_id,avatar_url,gravatar_id,url,html_url,followers_url,following_url,...,note,new_entry,date,employee_url,faculty,level1,level2,level3,level4,position
0,73616e646572,employees_github_usernames.csv,6605233.0,MDQ6VXNlcjY2MDUyMzM=,https://avatars.githubusercontent.com/u/660523...,NaN,https://api.github.com/users/73616e646572,https://github.com/73616e646572,https://api.github.com/users/73616e646572/foll...,https://api.github.com/users/73616e646572/foll...,...,0,False,2021-12-02,CFArends,Humanities,Humanities,"Student Affairs, Education and Research",General,None,Student Affairs Assistant
1,a-quei,ids_search_users.csv,6829836.0,MDQ6VXNlcjY4Mjk4MzY=,https://avatars.githubusercontent.com/u/682983...,NaN,https://api.github.com/users/a-quei,https://github.com/a-quei,https://api.github.com/users/a-quei/followers,https://api.github.com/users/a-quei/following{...,...,0,False,2021-12-02,GChen,Science,Science,Information and computing sciences,Intelligent Software Systems,Natural Language Processing,Lecturer
2,ajueling,ids_search_repos.csv,20947797.0,MDQ6VXNlcjIwOTQ3Nzk3,https://avatars.githubusercontent.com/u/209477...,NaN,https://api.github.com/users/AJueling,https://github.com/AJueling,https://api.github.com/users/AJueling/followers,https://api.github.com/users/AJueling/followin...,...,0,False,2021-12-02,AJuling2,UU,UU,None,None,None,None
3,albertgatt,employees_github_usernames.csv,8494566.0,MDQ6VXNlcjg0OTQ1NjY=,https://avatars.githubusercontent.com/u/849456...,NaN,https://api.github.com/users/albertgatt,https://github.com/albertgatt,https://api.github.com/users/albertgatt/followers,https://api.github.com/users/albertgatt/follow...,...,NaN,True,2021-12-02,AGatt,Science,Science,Information and computing sciences,Intelligent Software Systems,Natural Language Processing,Associate Professor
4,aldertzomer,ids_search_users.csv,10392411.0,MDQ6VXNlcjEwMzkyNDEx,https://avatars.githubusercontent.com/u/103924...,NaN,https://api.github.com/users/aldertzomer,https://github.com/aldertzomer,https://api.github.com/users/aldertzomer/follo...,https://api.github.com/users/aldertzomer/follo...,...,0,False,2021-12-02,ALZomer,Veterinary Medicine,Veterinary Medicine,Department Biomolecular Health Sciences,Infectious Diseases & Immunology,Clinical Infectiology,Assistant Professor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,wouter-swierstra,ids_search_repos.csv,1588245.0,MDQ6VXNlcjE1ODgyNDU=,https://avatars.githubusercontent.com/u/158824...,NaN,https://api.github.com/users/wouter-swierstra,https://github.com/wouter-swierstra,https://api.github.com/users/wouter-swierstra/...,https://api.github.com/users/wouter-swierstra/...,...,0,False,2021-12-02,WSSwierstra,Science,Science,Information and computing sciences,Intelligent Software Systems,Software Technology,Assistant Professor
150,wouterdroogers,ids_search_users.csv,38694532.0,MDQ6VXNlcjM4Njk0NTMy,https://avatars.githubusercontent.com/u/386945...,NaN,https://api.github.com/users/WouterDroogers,https://github.com/WouterDroogers,https://api.github.com/users/WouterDroogers/fo...,https://api.github.com/users/WouterDroogers/fo...,...,0,False,2021-12-02,WJDroogers,Science,Science,Biology,Biodynamics and Biocomplexity,Cell Biology,PhD Candidate
151,yuntaoj,ids_search_users.csv,74980142.0,MDQ6VXNlcjc0OTgwMTQy,https://avatars.githubusercontent.com/u/749801...,NaN,https://api.github.com/users/yuntaoj,https://github.com/yuntaoj,https://api.github.com/users/yuntaoj/followers,https://api.github.com/users/yuntaoj/following...,...,0,False,2021-12-02,YJi,Geosciences,Geosciences,Earth Sciences,Structural geology & EM,None,Researcher
152,zievathustra,ids_search_repos.csv,61464979.0,MDQ6VXNlcjYxNDY0OTc5,https://avatars.githubusercontent.com/u/614649...,NaN,https://api.github.com/users/zievathustra,https://github.com/zievathustra,https://api.github.com/users/zievathustra/foll...,https://api.github.com/users/zievathustra/foll...,...,0,False,2021-12-02,JASieverink,Un